In [3]:
import pandas as pd  # 导入 pandas 库

data = pd.read_csv('./data/USER_INFO_M.csv', index_col=0, encoding='gbk')  # 读取数据文件
print('原始数据形状为：', data.shape)  # 打印原始数据的形状

data_drop = pd.DataFrame.drop_duplicates(data, subset=None, keep='first', inplace=False)  # 删除重复的行
print('删除重复记录后的数据形状为：', data_drop.shape)  # 打印删除重复行后的数据形状

def FeatureEquals(df):  # 定义一个函数用于比较列是否完全相等
    dfEquals = pd.DataFrame([], columns=df.columns, index=df.columns)  # 创建一个空的 DataFrame 用于存储比较结果
    for i in df.columns:  # 遍历列名
        for j in df.columns:  # 再次遍历列名
            dfEquals.loc[i, j] = df.loc[:, i].equals(df.loc[:, j])  # 比较两列是否完全相等
    return dfEquals  # 返回比较结果的 DataFrame

detEquals = FeatureEquals(data_drop)  # 调用函数进行列比较
lenDet = detEquals.shape[0]  # 获取比较结果 DataFrame 的行数
dupCol = []  # 创建一个空列表用于存储重复的列名
for k in range(lenDet):  # 外层循环
    for l in range(k + 1, lenDet):  # 内层循环
        if detEquals.iloc[k, l] and (detEquals.columns[l] not in dupCol):  # 如果两列相等且列名未在重复列表中
            dupCol.append(detEquals.columns[l])  # 将列名添加到重复列表
data_drop.drop(dupCol, axis=1, inplace=True)  # 删除重复的列
print('删除重复列后的数据形状：', data_drop.shape)  # 打印删除重复列后的数据形状

# 数据降维
del data_drop['MODEL_NAME']  # 删除指定列
del data_drop['AGREE_EXP_DATE']  # 删除指定列
del data_drop['CUST_SEX']  # 删除指定列
del data_drop['CONSTELLATION_DESC']  # 删除指定列
del data_drop['CERT_AGE']  # 删除指定列
print('降维后的数据形状为:', data_drop.shape)  # 打印降维后的数据形状


原始数据形状为： (425688, 34)
删除重复记录后的数据形状为： (425673, 34)


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


删除重复列后的数据形状： (425673, 34)
降维后的数据形状为: (425673, 29)


In [4]:
naRate = (data_drop.isnull().sum() /  # 计算每列的缺失值数量
          data_drop.shape[0] * 100).astype('str') + '%'  # 计算缺失值占比，并转换为字符串加上'%'
print('data 每个特征缺失的率为：\n', naRate)  # 打印每个特征的缺失率

data 每个特征缺失的率为：
 USER_ID                      0.00023492211157390768%
INNET_MONTH                  0.00023492211157390768%
IS_AGREE                     0.00023492211157390768%
CREDIT_LEVEL                 0.00023492211157390768%
VIP_LVL                           37.32325047630458%
ACCT_FEE                     0.00023492211157390768%
CALL_DURA                    0.00023492211157390768%
NO_ROAM_LOCAL_CALL_DURA      0.00023492211157390768%
NO_ROAM_GN_LONG_CALL_DURA    0.00023492211157390768%
GN_ROAM_CALL_DURA            0.00023492211157390768%
CDR_NUM                      0.00023492211157390768%
NO_ROAM_CDR_NUM              0.00023492211157390768%
NO_ROAM_LOCAL_CDR_NUM        0.00023492211157390768%
NO_ROAM_GN_LONG_CDR_NUM      0.00023492211157390768%
GN_ROAM_CDR_NUM              0.00023492211157390768%
P2P_SMS_CNT_UP               0.00023492211157390768%
TOTAL_FLUX                   0.00023492211157390768%
LOCAL_FLUX                   0.00023492211157390768%
GN_ROAM_FLUX                 

In [5]:
data_drop['VIP_LVL'] = data_drop['VIP_LVL'].fillna(0)
data_drop['OS_DESC'] = data_drop['OS_DESC'].fillna('ANDROID')
print('处理缺失值后数据集的形状为：', data_drop.shape)

处理缺失值后数据集的形状为： (425673, 29)


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
data_drop.describe()

,INNET_MONTH,IS_AGREE,CREDIT_LEVEL,VIP_LVL,ACCT_FEE,CALL_DURA,NO_ROAM_LOCAL_CALL_DURA,NO_ROAM_GN_LONG_CALL_DURA,GN_ROAM_CALL_DURA,CDR_NUM,...,LOCAL_FLUX,GN_ROAM_FLUX,CALL_DAYS,CALLING_DAYS,CALLED_DAYS,CALL_RING,CALLING_RING,CALLED_RING,TERM_TYPE,IS_LOST
count,425672.000000,425672.000000,425672.000000,425673.000000,425672.00000,425672.000000,425672.000000,425672.000000,425672.000000,425672.000000,...,425672.000000,425672.000000,425672.000000,425672.000000,425672.000000,425672.000000,425672.000000,425672.000000,425672.000000,0.0
mean,34.067094,0.514356,66.009343,48.466964,120.95791,21887.222639,12801.741585,3547.150585,5466.478042,187.468687,...,783.449483,333.965681,23.657971,19.540534,20.170655,47.422156,29.442399,31.263628,3.760508,NaN
std,34.138366,0.499794,0.962192,48.964539,144.68319,24857.761328,17721.210296,6564.093791,13315.597788,212.761225,...,1774.567978,1017.944029,7.807703,8.712807,8.787368,55.587782,41.007111,33.536120,0.470423,NaN
min,-251.000000,0.000000,0.000000,0.000000,0.01000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,NaN
25%,9.000000,0.000000,65.000000,0.000000,56.00000,6075.750000,1802.000000,0.000000,0.000000,58.000000,...,60.157104,0.000000,20.000000,13.000000,14.000000,17.000000,10.000000,10.000000,4.000000,NaN
50%,24.000000,1.000000,66.000000,4.000000,88.08500,14403.000000,6828.000000,995.000000,310.000000,124.000000,...,372.846939,20.848515,27.000000,21.000000,22.000000,32.000000,19.000000,22.000000,4.000000,NaN
75%,48.000000,1.000000,67.000000,99.000000,146.00000,28803.000000,16782.000000,4402.000000,4717.000000,239.000000,...,926.450099,295.657941,29.000000,27.000000,28.000000,61.000000,36.000000,41.000000,4.000000,NaN
max,248.000000,1.000000,67.000000,99.000000,14713.39000,732638.000000,609545.000000,293483.000000,512448.000000,5706.000000,...,481358.912600,74192.874000,31.000000,31.000000,31.000000,4335.000000,4278.000000,1778.000000,4.000000,NaN


In [7]:
# 删除异常数据
data_drop = data_drop[data_drop['INNET_MONTH'] >= 0]
data_drop = data_drop[data_drop['ACCT_FEE'] < 400000]
print('处理异常值后数据集的形状为：', data_drop.shape)
data_drop.to_csv('./data/data_drop.csv', index=True, header='infer', encoding='utf8')

处理异常值后数据集的形状为： (425671, 29)


In [8]:
import pandas as pd
import numpy as np

data_drop = pd.read_csv('./data/data_drop.csv', encoding='utf8')
# 按照"USER_ID"对数据进行分组，并对指定列计算中位数和方差
data_group = data_drop.groupby("USER_ID").agg({
        'ACCT_FEE': [np.median, np.var],\
        'CALL_DURA': [np.median, np.var], 'CDR_NUM': [np.median, np.var],\
                  'NO_ROAM_LOCAL_CALL_DURA': [np.median, np.var],\
                  'NO_ROAM_LOCAL_CDR_NUM': [np.median, np.var],\
                  'NO_ROAM_GN_LONG_CALL_DURA': [np.median, np.var],\
                  'NO_ROAM_GN_LONG_CDR_NUM': [np.median, np.var],\
                  'GN_ROAM_CALL_DURA': [np.median, np.var], \
                  'GN_ROAM_CDR_NUM': [np.median, np.var], \
                  'NO_ROAM_CDR_NUM': [np.median, np.var],\
                  'P2P_SMS_CNT_UP': [np.median, np.var],
                  'TOTAL_FLUX': [np.median, np.var], \
                  'LOCAL_FLUX': [np.median, np.var], \
                  'GN_ROAM_FLUX': [np.median, np.var],\
                  'CALL_DAYS': [np.median, np.var], \
                  'CALLING_DAYS': [np.median, np.var],\
                  'CALLED_DAYS': [np.median, np.var],\
                  'CALL_RING': [np.median, np.var],\
                  'CALLING_RING': [np.median, np.var],\
                  'CALLED_RING': [np.median, np.var],\
                  'INNET_MONTH': [np.median, np.var], })

print('data_group的形状为:', data_group.shape)  # 打印数据组的形状
# 重命名列名
data_group.columns = ['ACCT_FEE_median', 'ACCT_FEE_var', 'CALL_DURA_median', 
                      'CALL_DURA_var', 'CDR_NUM_median', 'CDR_NUM_var', 
                      'NO_ROAM_LOCAL_CALL_DURA_median', 
                      'NO_ROAM_LOCAL_CALL_DURA_var', 
                      'NO_ROAM_LOCAL_CDR_NUM_median', 
                      'NO_ROAM_LOCAL_CDR_NUM_var', 
                      'NO_ROAM_GN_LONG_CALL_DURA_median', 
                      'NO_ROAM_GN_LONG_CALL_DURA_var', 
                      'NO_ROAM_GN_LONG_CDR_NUM_median', 
                      'NO_ROAM_GN_LONG_CDR_NUM_var', 
                      'GN_ROAM_CALL_DURA_median', 
                      'GN_ROAM_CALL_DURA_var', 
                      'GN_ROAM_CDR_NUM_median', 
                      'GN_ROAM_CDR_NUM_var', 
                      'NO_ROAM_CDR_NUM_median', 
                      'NO_ROAM_CDR_NUM_var', 
                      'P2P_SMS_CNT_UP_median', 'P2P_SMS_CNT_UP_var', 
                      'TOTAL_FLUX_median', 'TOTAL_FLUX_var', 
                      'LOCAL_FLUX_median', 'LOCAL_FLUX_var', 
                      'GN_ROAM_FLUX_median', 'GN_ROAM_FLUX_var', 
                      'CALL_DAYS_median', 'CALL_DAYS_var', 
                      'CALLING_DAYS_median', 'CALLING_DAYS_var', 
                      'CALLED_DAYS_median', 'CALLED_DAYS_var', 
                      'CALL_RING_median', 'CALL_RING_var', 
                      'CALLING_RING_median', 'CALLING_RING_var', 
                      'CALLED_RING_median', 'CALLED_RING_var', 
                      'INNET_MONTH_median', 'INNET_MONTH_var']
# # 将处理后的数据保存到 CSV 文件
data_group.to_csv('./data/data_group.csv', index=True, header='infer', encoding='utf8')

data_group的形状为: (299823, 42)


In [10]:
# 将每个用户三个月的合约是否有效，合并为一条记录
# 定义合并合约有效记录函数
def fun1(data):
    """
    该函数用于处理每个用户三个月的合约是否有效数据
    参数:     data (DataFrame): 包含用户合约是否有效信息的数据
    返回:     处理后的结果，根据不同条件返回不同值
    """
    if data.shape[0]!= 3:  # 如果数据行数不为 3
        return 0
    elif sum(data.iloc[:, 1] == 1) == 3:  # 如果三个月的合约都有效
        return 1.5
    else:  # 其他情况
        return data.iloc[-1, 1] - data.iloc[:2, 1].mean()

# 对名为data_drop的数据框进行操作，并生成一个新的数据框data_agree
data_agree = data_drop[["USER_ID","IS_AGREE"]].groupby("USER_ID").apply(lambda x: fun1(x))
# 这里按照"USER_ID"对数据进行分组，并应用 fun1 函数进行处理，结果存储在 data_agree 中
print('data_agree的形状为:', data_agree.shape)
print(data_agree.head(),'\n')
data_agree.to_csv('./data/data_agree.csv', index=True, header='infer', encoding='utf8')

data_agree的形状为: (299823,)
USER_ID
U3114031824148707    0
U3114031824148874    0
U3114031824148975    0
U3114031824149138    0
U3114031824149150    0
dtype: int64 



In [11]:
# 将每个用户三个月的 VIP 等级合并为一条记录
def fun2(data):
    """
    该函数用于处理每个用户三个月的 VIP 等级数据
    参数:     data (DataFrame): 包含用户 VIP 等级信息的数据
    返回:     处理后的结果，根据不同条件返回不同值
    """
    if data.shape[0]!= 3:  # 如果数据行数不为 3
        return 0
    elif (data.iloc[0, 1] == data.iloc[1, 1]) & (data.iloc[0, 1] == data.iloc[2, 1]):  # 如果三个月的 VIP 等级相同
        return data.iloc[2, 1]
    else:  # 其他情况
        return data.iloc[2, 1] - data.iloc[:2, 1].mean()

data_vip = data_drop[['USER_ID','VIP_LVL']].groupby('USER_ID').apply(lambda x: fun2(x))
# 这里按照"USER_ID"对数据进行分组，并应用 fun2 函数进行处理，结果存储在 data_vip 中
print('data_vip的形状为:', data_vip.shape)
print(data_vip.head(),'\n\n')
data_vip.to_csv('./data/data_vip.csv', index=True, header='infer', encoding='utf8')

data_vip的形状为: (299823,)
USER_ID
U3114031824148707    0
U3114031824148874    0
U3114031824148975    0
U3114031824149138    0
U3114031824149150    0
dtype: int64 




In [12]:
# 取每个用户三个月信用等级的平均数作为一行记录
data_credit = data_drop.groupby('USER_ID').agg({'CREDIT_LEVEL': np.mean, })
data_credit.iloc[:10]
print('data_credit的形状为:', data_credit.shape)
print(data_credit.head())
data_credit.to_csv('./data/data_credit.csv', index=True, header='infer', encoding='utf8')

data_credit的形状为: (299823, 1)
                   CREDIT_LEVEL
USER_ID                        
U3114031824148707          67.0
U3114031824148874          65.0
U3114031824148975          65.0
U3114031824149138          65.0
U3114031824149150          65.0


In [13]:
# 简化手机品牌
string = ['苹果', '小米', '华为', '三星', '诺基亚', '联想', 'LG']

def Replace(x=None, string=string):
    if x not in string:
        x = '其他'
    return x

# 每个ID的手机品牌只取第一个月的
data_str = data_drop.groupby("USER_ID").apply(lambda x: x.iloc[0])
data_manu = data_str['MANU_NAME'].apply(Replace)
print('data_manu的形状为:', data_manu.shape)
print(data_manu.head())

# 简化操作系统
# 每个ID的手机操作系统也只取第一个月的
data_id = data_drop.groupby("USER_ID").apply(lambda x: x.iloc[0])
data_os = data_id["OS_DESC"].str.extract("([A-Z]+)")  # 保留所有的字母
print('data_os的形状为:', data_os.shape)
print(data_os.head())
data_manu.to_csv('./data/data_manu.csv', index=True, header='infer', encoding='utf8')
data_os.to_csv('./data/data_os.csv', index=True, header='infer', encoding='utf8')

In [ ]:
# 手机品牌独热编码
import pandas as pd  # 导入 pandas 库

data_manu = pd.read_csv('./data/data_manu.csv', encoding='utf8')  # 读取手机品牌数据文件
data_manu.index = data_manu.iloc[:, 0]  # 设置索引
data_manu = data_manu.drop(columns='USER_ID')  # 删除'USER_ID'列

data_os = pd.read_csv('./data/data_os.csv', encoding='utf8')  # 读取操作系统数据文件
data_os.index = data_os.iloc[:, 0]  # 设置索引
data_os = data_os.drop(columns='USER_ID')  # 删除'USER_ID'列

data_drop = pd.read_csv('./data/data_drop.csv', encoding='utf8')  # 读取其他相关数据文件

data_group = pd.read_csv('./data/data_group.csv', encoding='utf8')  # 读取分组数据文件
data_group.index = data_group.iloc[:, 0]  # 设置索引
data_group = data_group.drop(columns='USER_ID')  # 删除'USER_ID'列

data_agree = pd.read_csv('./data/data_agree.csv', encoding='utf8')  # 读取同意相关数据文件
data_agree.index = data_agree.iloc[:, 0]  # 设置索引
data_agree = data_agree.drop(columns='USER_ID')  # 删除'USER_ID'列

data_credit = pd.read_csv('./data/data_credit.csv', encoding='utf8')  # 读取信用数据文件
data_credit.index = data_credit.iloc[:, 0]  # 设置索引
data_credit = data_credit.drop(columns='USER_ID')  # 删除'USER_ID'列

data_vip = pd.read_csv('./data/data_vip.csv', encoding='utf8')  # 读取 VIP 数据文件
data_vip.index = data_vip.iloc[:, 0]  # 设置索引
data_vip = data_vip.drop(columns='USER_ID')  # 删除'USER_ID'列

data_manu = pd.get_dummies(data_manu)  # 对手机品牌数据进行独热编码
print('独热编码后的手机品牌的形状：', data_manu.shape)  # 打印形状
print(data_manu.head())  # 打印前几行数据
data_manu.to_csv('./data/data_manu.csv', encoding='utf8')  # 保存编码后的数据

# 操作系统独热编码
data_os = pd.get_dummies(data_os)  # 对操作系统数据进行独热编码
print('独热编码后的操作系统的形状：', data_os.shape)  # 打印形状
print(data_os.head(),'\n\n')  # 打印前几行数据
data_os.to_csv('./data/data_os.csv', encoding='utf8')  # 保存编码后的数据

独热编码后的手机品牌的形状： (299968, 8)
                   MANU_NAME_LG  MANU_NAME_三星  MANU_NAME_其他  MANU_NAME_华为  \
USER_ID                                                                     
U3114031824148707             0             0             0             0   
U3114031824148874             0             0             0             0   
U3114031824148975             0             0             0             0   
U3114031824149138             0             0             0             0   
U3114031824149150             0             1             0             0   

                   MANU_NAME_小米  MANU_NAME_联想  MANU_NAME_苹果  MANU_NAME_诺基亚  
USER_ID                                                                     
U3114031824148707             0             0             1              0  
U3114031824148874             0             0             1              0  
U3114031824148975             0             0             1              0  
U3114031824149138             0             0   

In [ ]:
print('data_drop的形状：', data_drop.shape)
print(' data_group的形状：', data_group.shape)
print(' data_agree的形状：', data_agree.shape)
print(' data_vip的形状：', data_vip.shape)
print(' data_credit的形状：', data_credit.shape)
print(' data_manu的形状：', data_manu.shape)
print(' data_os的形状：', data_os.shape,'\n\n')

import pandas as pd  # 导入 pandas 库

# 将多个数据集沿列方向合并
data_preprocessed = pd.concat([data_group, data_agree, data_vip, data_credit,data_manu, data_os, ], axis=1)
# 打印合并后数据集的形状
print('合并后数据集的形状为:', data_preprocessed.shape)

# 为合并后的数据集列重命名
data_preprocessed.columns = ['ACCT_FEE_median', 'ACCT_FEE_var',
                             'CALL_DURA_median', 'CALL_DURA_var',
                             'CDR_NUM_median', 'CDR_NUM_var',
                             'NO_ROAM_LOCAL_CALL_DURA_median',
                             'NO_ROAM_LOCAL_CALL_DURA_var',
                             'NO_ROAM_LOCAL_CDR_NUM_median',
                             'NO_ROAM_LOCAL_CDR_NUM_var',
                             'NO_ROAM_GN_LONG_CALL_DURA_median',
                             'NO_ROAM_GN_LONG_CALL_DURA_var',
                             'NO_ROAM_GN_LONG_CDR_NUM_median',
                             'NO_ROAM_GN_LONG_CDR_NUM_var',
                             'GN_ROAM_CALL_DURA_median',
                             'GN_ROAM_CALL_DURA_var',
                             'GN_ROAM_CDR_NUM_median',
                             'GN_ROAM_CDR_NUM_var',
                             'NO_ROAM_CDR_NUM_median',
                             'NO_ROAM_CDR_NUM_var',
                             'P2P_SMS_CNT_UP_median',
                             'P2P_SMS_CNT_UP_var',
                             'TOTAL_FLUX_median', 'TOTAL_FLUX_var',
                             'LOCAL_FLUX_median', 'LOCAL_FLUX_var',
                             'GN_ROAM_FLUX_median', 'GN_ROAM_FLUX_var',
                             'CALL_DAYS_median', 'CALL_DAYS_var',
                             'CALLING_DAYS_median', 'CALLING_DAYS_var',
                             'CALLED_DAYS_median', 'CALLED_DAYS_var',
                             'CALL_RING_median', 'CALL_RING_var',
                             'CALLING_RING_median', 'CALLING_RING_var',
                             'CALLED_RING_median', 'CALLED_RING_var',
                             'INNET_MONTH_median', 'INNET_MONTH_var',
                             'IS_AGREE', 'VIP_LVL', 'CREDIT_LEVEL',
                             'LG', '三星', '其他', '华为', '小米',
                             '联想', '苹果', '诺基亚',
                             '0_ANDROID', '0_BADA', '0_BB', '0_BLACKBERRY',
                             '0_IOS', '0_LINUX', '0_WINDOWS']
# 打印合并后数据集的前几行
print(data_preprocessed.head())
# 将合并后的数据集保存为 CSV 文件
data_preprocessed.to_csv('./data/data_preprocessed.csv', encoding='utf8')

data_drop的形状： (899901, 30)
 data_group的形状： (299967, 42)
 data_agree的形状： (299967, 1)
 data_vip的形状： (299967, 1)
 data_credit的形状： (299967, 1)
 data_manu的形状： (299968, 8)
 data_os的形状： (299968, 7) 


合并后数据集的形状为: (299968, 60)
                   ACCT_FEE_median  ACCT_FEE_var  CALL_DURA_median  \
U3114031824148707             76.0     27.907500           11901.0   
U3114031824148874            260.3   2889.998633           22991.0   
U3114031824148975            166.0      0.003333           18972.0   
U3114031824149138            146.2   2150.297500           42921.0   
U3114031824149150             77.3     48.823333            1206.0   

                   CALL_DURA_var  CDR_NUM_median   CDR_NUM_var  \
U3114031824148707   1.640320e+07           183.0  11849.333333   
U3114031824148874   2.862449e+07           373.0   6857.333333   
U3114031824148975   4.498497e+07           184.0   1849.333333   
U3114031824149138   3.123659e+07           217.0    964.000000   
U3114031824149150   1.721161e+

In [ ]:
# 导入所需的库
import pandas as pd  # 用于数据处理和分析
from sklearn.model_selection import train_test_split  # 用于划分数据集
from sklearn.preprocessing import StandardScaler  # 用于数据标准化
from sklearn.neural_network import MLPClassifier  # 多层感知机分类器
from sklearn.metrics import classification_report  # 用于生成分类报告

# 导入数据
data_drop = pd.read_csv('./data/data_drop.csv', encoding='utf-8', index_col=0)  # 读取 data_drop.csv 数据文件
data_preprocessed = pd.read_csv('./data/data_preprocessed.csv', encoding='utf-8', index_col=0)  # 读取 data_preprocessed.csv 数据文件

# 取 data_preprocessed 作为输入，取 data_drop 中三月份的数据的目标变量作为输出
data_target = data_drop.loc[:, ['USER_ID', 'IS_LOST']]  # 提取特定列作为目标变量
data_target = data_target.loc[data_target["USER_ID"].isin(data_preprocessed.index)]  # 筛选出 USER_ID 在 data_preprocessed 索引中的数据
data_target = data_target.loc[201603].drop_duplicates()  # 筛选出三月份的数据并去除重复项
print('目标变量数据集的形状为:', data_target.shape)  # 打印目标变量数据集的形状

# 划分数据集
x = data_preprocessed  # 输入特征数据
y = data_target  # 目标变量
x_train, x_test, y_train, y_test = train_test_split(x, y['IS_LOST'], test_size=0.2, random_state=42)  # 以 8:2 的比例划分训练集和测试集，设置随机数种子为 42
print('训练集数据的形状为：', x_train.shape)  # 打印训练集数据的形状
print('训练集标签的形状为：', y_train.shape)  # 打印训练集标签的形状
print('测试集数据的形状为：', x_test.shape)  # 打印测试集数据的形状
print('测试集标签的形状为：', y_test.shape)  # 打印测试集标签的形状

# 数据标准化
stdScaler = StandardScaler().fit(x_train)  # 在训练集上拟合标准化器
x_stdtrain = stdScaler.transform(x_train)  # 对训练集数据进行标准化
x_stdtest = stdScaler.transform(x_test)  # 对测试集数据进行标准化
print('标准化后的 x_stdtrain:\n', x_stdtrain)  # 打印标准化后的训练集数据
print('标准化后的 x_stdtest:\n', x_stdtest)  # 打印标准化后的测试集数据

KeyError: 201603

In [ ]:
# 建立模型
bpnn = MLPClassifier(hidden_layer_sizes=(17, 10),\
                     max_iter=200, solver='lbfgs', random_state=50)
bpnn.fit(x_stdtrain, y_train)
print('构建的模型为:\n', bpnn)

In [ ]:
# 模型预测
y_pre = bpnn.predict(x_stdtest)
print('多层感知器预测结果评价报告：\n', classification_report(y_test, y_pre))

# %matplotlib inline
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# 绘制ROC曲线图
plt.rcParams['font.sans-serif'] = 'SimHei'  # 显示中文
plt.rcParams['axes.unicode_minus'] = False  # 显示负号
fpr, tpr, thresholds = roc_curve(y_pre, y_test)  # 求出TPR和FPR
plt.figure(figsize=(6, 4))  # 创建画布
plt.plot(fpr, tpr)  # 绘制曲线
plt.title('用户流失模型的ROC曲线')  # 标题
plt.xlabel('FPR')  # x轴标签
plt.ylabel('TPR')  # y轴标签
plt.show()  # 显示图形
plt.close